In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sys

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'
# Load training datasets and testing datasets 
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_12244/4005976825.py:20: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
def filter(dataset): 
    # Get only the rows that contain state (PAR, ACC, ECO, CON, FIN, INT, REQ, RST) and proto (igmp, arp, icmp, udp, tcp, ipv6-icmp, rarp)
    dataset = dataset[dataset['state'].isin(['PAR', 'ACC', 'ECO', 'CON', 'FIN', 'INT', 'REQ', 'RST'])]
    dataset = dataset[dataset['proto'].isin(['igmp', 'arp', 'icmp', 'udp', 'tcp', 'ipv6-icmp', 'rarp'])]
    return dataset

In [4]:
test_basic = filter(test_basic)
test_plus = filter(test_plus)

### test_complete

In [11]:
# Merge 'normal' traffic from test_basic and test_plus
test_basic_n = test_basic[test_basic['label'] == 0]
test_plus_n = test_plus[test_plus['label'] == 0]
test_n = pd.concat([test_basic_n, test_plus_n])

In [14]:
# Get all attack from test_basic
test_ka = test_basic[test_basic['label'] == 1]

In [22]:
# Get 71685 samples from test_n
test_n = test_n.sample(n=71685)

In [24]:
test_ua = test_plus[test_plus['label'] == 1]

In [26]:
test_ua = test_ua.sample(n=7168)

In [27]:
test_complete = pd.concat([test_ka, test_n, test_ua])

In [28]:
# save test_complete to csv
test_complete.to_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv', index=False)

### test_cat

In [8]:
test_cat = test_basic[test_basic['label'] == 1]

In [9]:
# save test_complete to csv
test_cat.to_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv', index=False)

### test_cat_eq

In [6]:
test_basic['attack_cat'].value_counts()

normal            72400
generic           53677
exploits           6948
reconnaissance     2942
dos                 949
Name: attack_cat, dtype: int64

In [10]:
gen = test_basic[test_basic['attack_cat'] == 'generic'].sample(n = 949)
ex = test_basic[test_basic['attack_cat'] == 'exploits'].sample(n = 949)
rec = test_basic[test_basic['attack_cat'] == 'reconnaissance'].sample(n = 949)
dos = test_basic[test_basic['attack_cat'] == 'dos']


In [11]:
test_cat_eq = pd.concat([gen, ex, rec, dos])

In [14]:
test_cat_eq['attack_cat'].value_counts()

generic           949
exploits          949
reconnaissance    949
dos               949
Name: attack_cat, dtype: int64

In [12]:
# save test_complete to csv
test_cat_eq.to_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv', index=False)